<a href="https://colab.research.google.com/github/LucaNicoleta/Clickbait-Spoiling/blob/main/models/synopsis_generation/QA_model_phrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install evaluate transformers[sentencepiece]
!pip install accelerate tensorflow tqdm dataset numpy

!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [ ]:
from tqdm.auto import tqdm
import collections
import numpy as np
import evaluate

metric = evaluate.load("squad")

n_best=20

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [ ]:

import pandas as pd

# citim json si il stocam intr-un dataframe
df_test = pd.read_json("/content/drive/MyDrive/click-data/validation.jsonl", lines=True)
# print(pd.concat([df_test["targetParagraphs"],df_test["postText"]],axis=1))

# coloana tags contine moment array cu un singur element asa ca desfacem array-ul
df_test["tags"] = list(map(lambda x: x[0], df_test["tags"].tolist()))
# etichetam valorile din tags cu valori numerice
df_test["tags"] = df_test["tags"].apply(lambda x: 0 if x == 'phrase' else 1 if x == 'passage' else 2)

# print(df_test["tags"])
# df_test = df_test.loc[df_test["tags"]!=2]
df_test["lung_par"] = df_test["targetParagraphs"].apply(lambda x: [len(i) for i in x])

df_test["nrPar"] = df_test["targetParagraphs"].apply(lambda x: len(x))

# desfacem array-ul de stringuri din coloana ce contine paragrafele
df_test["targetParagraphs"] = df_test["targetParagraphs"].apply(lambda x: " ".join(x))
df_test["postText"] = df_test["postText"].apply(lambda x: " ".join(x))
df_test["allText"] = df_test[["targetTitle", "targetParagraphs"]].apply(" ".join, axis=1)

# prelucram datele de antrenare repetand pasii de mai sus
df = pd.read_json("/content/drive/MyDrive/click-data/train.jsonl", lines=True)
df["tags"] = list(map(lambda x: x[0], df["tags"].tolist()))

df["tags"] = df["tags"].apply(lambda x: 0 if x == 'phrase' else 1 if x == 'passage' else 2)
df["lung_par"] = df["targetParagraphs"].apply(lambda x: [len(i) for i in x])
df["nrPar"] = df["targetParagraphs"].apply(lambda x: len(x))
df["targetParagraphs"] = df["targetParagraphs"].apply(lambda x: " ".join(x))
df["postText"] = df["postText"].apply(lambda x: " ".join(x))
df["allText"] = df[["targetTitle", "targetParagraphs"]].apply(" ".join, axis=1)

In [ ]:
from datasets import Dataset
def get_real_position(di):
    #print(di["spoilerPositions"][0])
    ps = di["spoilerPositions"][0][0][0]
    cs = di["spoilerPositions"][0][0][1]
    if ps<0:
        if (di["allText"][cs:(cs + 10)] != di["spoiler"][0][:10]):
            pass
            #print("$$$$  ", di["uuid"])
        return cs
    if di["lung_par"][ps]<cs:
        cs+=len(di["targetTitle"])
        #print(di["allText"][cs:(cs + 10)])
        #print(di["spoiler"][0])
        if (di["allText"][cs:(cs + 10)] != di["spoiler"][0][:10]):
            pass
            #print("$$$$  ", di["uuid"])
        return cs
    #cf = di["spoilerPositions"][0][1][1]
    #lung_sp = cf-cs
    #print("Lung spoiler:",lung_sp)
    #print(ps," ",cs," ")
    s = sum(di["lung_par"][0:ps])+ps+cs+len(di["targetTitle"])+1
    #print(s)
    #print(di["allText"][s:(s+10)])
    #print(di["spoiler"][0])
    if(di["allText"][s:(s+10)]!=di["spoiler"][0][:10]):
        pass
        #print("$$$$  ",di["uuid"])
    return s




def transform_df_in_dict(old):
    new_dict = []
    for d in old:
        nd = { 
            "id": d["uuid"],
            "context": d["allText"],
            "question": d["postText"],
            "answers": 
                        {
                            "text": [d["spoiler"][0]],
                            "answer_start": [get_real_position(d)]
                        }
                    
        }
        new_dict.append(nd)
    return new_dict

phrase_df = df[df['tags']==1]

a = phrase_df.to_dict('records')
raw_train_data= Dataset.from_list(transform_df_in_dict(a))
print(raw_train_data[0])

phrase_df_test = df_test[df_test['tags']==1]

a = phrase_df_test.to_dict('records')
raw_test_data= Dataset.from_list(transform_df_in_dict(a))

{'id': '0af11f6b-c889-4520-9372-66ba25cb7657', 'context': 'Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had A Better Idea It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure.

In [ ]:
max_length = 384
stride = 128
from datasets import Dataset
from transformers import AutoTokenizer
model_checkpoint = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

train_dataset = raw_train_data.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_train_data.column_names,
)
print(len(raw_train_data), len(train_dataset))

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--deepset--roberta-base-squad2/snapshots/d39b8d4166b0683451bbce6f047de1a238c0b5bf/config.json
Model config RobertaConfig {
  "_name_or_path": "deepset/roberta-base-squad2",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "name": "Roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /ro

  0%|          | 0/2 [00:00<?, ?ba/s]

1274 3639


In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

validation_dataset = raw_test_data.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_test_data.column_names,
)
print(len(raw_test_data), len(validation_dataset))


  0%|          | 0/1 [00:00<?, ?ba/s]

322 971


In [ ]:
from transformers import Trainer, AutoTokenizer, TrainingArguments
import pickle
from transformers import AutoModelForQuestionAnswering, AutoConfig

model_checkpoint = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
config = AutoConfig.from_pretrained('deepset/roberta-base-squad2')
model = AutoModelForQuestionAnswering.from_config(config)
def model_init():
    return AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

args = TrainingArguments(
            output_dir='checkpoints',
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(

    model_init=model_init,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
pickle.dump(trainer.model, open('task2_passage.sav', 'wb'))
compute_metrics(start_logits, end_logits, validation_dataset, raw_test_data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--deepset--roberta-base-squad2/snapshots/d39b8d4166b0683451bbce6f047de1a238c0b5bf/config.json
Model config RobertaConfig {
  "_name_or_path": "deepset/roberta-base-squad2",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "name": "Roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /ro

Epoch,Training Loss,Validation Loss
1,No log,No log
2,1.664000,No log
3,1.257700,No log


The following columns in the evaluation set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 971
  Batch size = 8
Saving model checkpoint to checkpoints/checkpoint-500
Configuration saved in checkpoints/checkpoint-500/config.json
Model weights saved in checkpoints/checkpoint-500/pytorch_model.bin
tokenizer config file saved in checkpoints/checkpoint-500/tokenizer_config.json
Special tokens file saved in checkpoints/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this 

  0%|          | 0/322 [00:00<?, ?it/s]

{'exact_match': 16.149068322981368, 'f1': 44.42812821544332}

In [ ]:
import pickle
from transformers import pipeline
import tensorflow as tf
filename = 'task2_phrase.sav'
model = pickle.load(open(filename, 'rb'))
question_answerer = pipeline("question-answering", model=model, tokenizer='deepset/roberta-base-squad2', device=0)

for i in range(10):
    print(question_answerer(question=df["postText"].tolist()[i], context=df["allText"].tolist()[i]))

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--deepset--roberta-base-squad2/snapshots/d39b8d4166b0683451bbce6f047de1a238c0b5bf/config.json
Model config RobertaConfig {
  "_name_or_path": "deepset/roberta-base-squad2",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "name": "Roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /ro

{'score': 0.030374860391020775, 'start': 793, 'end': 804, 'answer': 'we go throw'}
{'score': 0.8675529360771179, 'start': 65, 'end': 69, 'answer': '2070'}
{'score': 0.3314509987831116, 'start': 226, 'end': 251, 'answer': 'intellectually stimulated'}
{'score': 0.007132420316338539, 'start': 484, 'end': 503, 'answer': 'Follow your passion'}
{'score': 0.5034038424491882, 'start': 772, 'end': 783, 'answer': 'rice cooker'}
{'score': 0.41447725892066956, 'start': 628, 'end': 655, 'answer': "you'll have to buy new ones"}
{'score': 0.323740154504776, 'start': 780, 'end': 820, 'answer': 'longstanding superstition or "kabbalah",'}
{'score': 0.5383720993995667, 'start': 223, 'end': 238, 'answer': 'rainbow colours'}
{'score': 0.10260897874832153, 'start': 25, 'end': 36, 'answer': '5/5 say yes'}
{'score': 0.43492141366004944, 'start': 1294, 'end': 1301, 'answer': 'Tapatío'}
